<a href="https://colab.research.google.com/github/hhk54250/IS20/blob/master/Option%20value%20using%20fast%20fourier%20transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np

def BSM_characteristic_function(v, x0, T, r, sigma):
    cf_value = np.exp(((x0 / T + r - 0.5 * sigma ** 2) * 1j * v
                - 0.5 * sigma ** 2 * v ** 2) * T)
    return cf_value
def BSM_call_characteristic_function(v,alpha, x0, T, r, sigma):
    res=np.exp(-r*T)/((alpha+1j*v)*(alpha+1j*v+1))\
        *BSM_characteristic_function((v-(alpha+1)*1j), x0, T, r, sigma)
    return res
    
def SimpsonW(N,eta):
    delt = np.zeros(N, dtype=np.float)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = eta*(3 + (-1) ** j - delt) / 3
    return SimpsonW
    

def Simposon_numerical_integrate(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N=400000
    B=100
    eta=B/N
    W=SimpsonW(N,eta)
    
    alpha=1.5
    sumx=0
    for j in range(N):
        v_j=j*eta
        temp=np.exp(-1j*v_j*k)*\
            BSM_call_characteristic_function(v_j,alpha, x0, T, r, sigma)*\
            W[j]            
        sumx+=temp.real

        
    return sumx*np.exp(-alpha*k)/np.pi

In [40]:
S0 = 100.0  # index level
K = 110.0  # option strike
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility

print (Simposon_numerical_integrate(S0, K, T, r, sigma))

5.943273183453813


In [41]:
%cd~

!git clone https://github.com/hhk54250/20MA573-HHK.git 
pass


/root
fatal: destination path '20MA573-HHK' already exists and is not an empty directory.


In [42]:

%cd 20MA573-HHK/src/
%ls

/root/20MA573-HHK/src
bsm.py  optiondata.dat  prj01.ipynb  prj02.ipynb  __pycache__/


In [43]:
from bsm import *


'''===============
Test bsm_price
================='''
gbm1 = Gbm(
    init_state = 100., 
    drift_ratio = .0475,
    vol_ratio = .2)
option1 = VanillaOption(
    otype = 1,
    strike = 110.,                
    maturity = 1.
)    

print('>>>>>>>>>>call value is ' + str(gbm1.bsm_price(option1)))

>>>>>>>>>>call value is 5.943273183452838


In [65]:
def fft(FFTFunc):
    N=2**10
    eta=0.15
    lambda_ = 2 * np.pi / (N *eta)
    m=3089
    sumy=0
    for t in range(N):
        sum1=np.exp(-1j*lambda_*eta*(t-1)*(m-1))*FFTFunc[t]
        sumy=sum1.real+sumy

        
    return sumy

def BSM_call_value_FFT(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N =2**10
    alpha=1.5
    
    eta=0.15
    lambda_ = 2 * np.pi / (N *eta)
    beta=x0-lambda_*N/2
    km=np.asarray([beta+i*lambda_ for i in range(N)])
    W=SimpsonW(N,eta)
    v=np.asarray([i*eta for i in range(N)])
    Psi=np.asarray([BSM_call_characteristic_function(vj,alpha, x0, T, r, sigma)  for vj in v])
    FFTFunc=Psi*np.exp(-1j*beta*v)*W
    
    
    y=fft(FFTFunc).real
    
    
    cT=np.exp(-alpha*km)*y/np.pi
    
    return cT

In [66]:
S0 = 100.0  # index level
K = 110.0  # option strike
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility
BSM_call_value_FFT(S0, K, T, r, sigma)

array([-1.20554780e+13, -1.13380002e+13, -1.06632228e+13, ...,
       -7.47482939e-15, -7.02996736e-15, -6.61158116e-15])

In [104]:
def BSM_call_value_FFT(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N =2**12
    alpha=1.5
    
    eta=0.25
    lambda_ = 2 * np.pi / (N *eta)
    beta=lambda_*N/2
    km=np.asarray([beta+i*lambda_ for i in range(N)])
    W=SimpsonW(N,eta)
    v=np.asarray([i*eta for i in range(N)])
    Psi=np.asarray([BSM_call_characteristic_function(vj,alpha, x0, T, r, sigma)  for vj in v])
    FFTFunc=Psi*np.exp(1j*beta*v)*W
    return FFTFunc
S0 = 100.0  # index level
K = 110.0  # option strike
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility
FFTFunc=BSM_call_value_FFT(S0, K, T, r, sigma)

t=0
alpha=1.5
N=2**12
eta=0.25
lambda_ = 2 * np.pi / (N *eta)
m=2814
sumy=0
for t in range(N):
    sum1=FFTFunc[t]*np.exp(-1j*lambda_*eta*t*m)
    sumy=sum1.real+sumy
sumy
cT=np.exp(-alpha*4.700480365792)*sumy/np.pi
cT

5.954032841836237